# 기계학습 - 타이타닉 데이터 분석 및 머신러닝 모형 구축 과제

In [1]:
# README.md도 만들고
# nbviewer?도 사용하기

# 과제 점수는  accuracy가 아닌 dataset를 분석하여 어떤 통찰력을 가졌는지에 대한 부분

#### 데이터 정의
- Survival:	생존 여부 (0 = No, 1 = Yes)
- Pclass: 티켓 등급 (1 = 1st, 2 = 2nd, 3 = 3rd)
- Name: 이름
- Sex: 성별
- Age: 나이
- Sibsp: 타이타닉에 탑승한 형제 / 배우자의 수
- Parch: 타이타닉에 탑승한 부모 / 자녀의 수
- Ticket: 티켓 번호
- Fare: 승객 요금
- Cabin: 객실 번호
- Embarked: 기항지 위치	(C = Cherbourg, Q = Queenstown, S = Southampton)

| Key | 정의 | 비고 |
|-----|------|-----|
| Survival | 생존 여부 | (0 = No, 1 = Yes) |
| Pclass | 티켓 등급 | (1 = 1st, 2 = 2nd, 3 = 3rd) |
| Name | 이름 | |
| Sex | 성별 | |
| Age | 나이 | |
| Sibsp | 타이타닉에 탑승한 형제 / 배우자의 수 | |
| Parch | 타이타닉에 탑승한 부모 / 자녀의 수 | |
| Ticket | 티켓 번호 | |
| Fare | 승객 요금 | |
| Cabin | 객실 번호 | |
| Embarked | 기항지 위치 | C = Cherbourg, Q = Queenstown, S = Southampton |

#### 훈련 데이터 분석

In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [11]:
print(train_df.describe(include='all'))

        PassengerId    Survived      Pclass                 Name   Sex  \
count    891.000000  891.000000  891.000000                  891   891   
unique          NaN         NaN         NaN                  891     2   
top             NaN         NaN         NaN  Dooley, Mr. Patrick  male   
freq            NaN         NaN         NaN                    1   577   
mean     446.000000    0.383838    2.308642                  NaN   NaN   
std      257.353842    0.486592    0.836071                  NaN   NaN   
min        1.000000    0.000000    1.000000                  NaN   NaN   
25%      223.500000    0.000000    2.000000                  NaN   NaN   
50%      446.000000    0.000000    3.000000                  NaN   NaN   
75%      668.500000    1.000000    3.000000                  NaN   NaN   
max      891.000000    1.000000    3.000000                  NaN   NaN   

               Age       SibSp       Parch  Ticket        Fare Cabin Embarked  
count   714.000000  891.000000 

#### 테스트 데이터 분석

In [5]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#### 기초 데이터 분석